# Propojení RAG databáze s fine-tuned modelem

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from transformers import pipeline

# --- KONFIGURACE (z předchozích kroků) ---
BASE_MODEL_PATH = "mistralai/Mistral-7B-Instruct-v0.2" 
ADAPTER_PATH = "./results/checkpoint-400" # Cesta k vašemu fine-tunovanému adaptéru
PERSIST_DB_DIR = "./rag_db"
EMBEDDING_MODEL_NAME = "all-mpnet-base-v2"

In [ ]:
# 1. Načtení jazykového modelu (stejně jako v chat.py)
# ---------------------------------------------------
print("Načítám jazykový model (LLM)...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH, quantization_config=bnb_config, device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH)
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()
print(" LLM úspěšně načten.")

In [ ]:
# 2. Načtení vektorové databáze
# ----------------------------
print("Načítám vektorovou databázi...")
device = "cuda" if torch.cuda.is_available() else "cpu"
embeddings = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME, model_kwargs={'device': device}
)
db = Chroma(persist_directory=PERSIST_DB_DIR, embedding_function=embeddings)
print("✅ Databáze úspěšně načtena.")

# Vytvoření RAG řetězce

# Vytvoření text-generation pipeline
text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    repetition_penalty=1.1
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Vytvoření retrieveru z databáze
# Tento objekt bude zodpovědný za vyhledávání v databázi
retriever = db.as_retriever(search_kwargs={"k": 4}) # Chceme 4 nejrelevantnější chunky

# Vytvoření finálního RAG řetězce (chain)
# "stuff" je jednoduchá metoda, která vezme nalezené chunky a vloží je do promptu
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True # Chceme vidět, z jakých zdrojů model čerpal
)

print(" RAG řetězec je připraven k použití.")

In [ ]:
# --- Položte zde svůj dotaz ---
query = "Jaké jsou hlavní postuláty kvantové mechaniky?" 
print(f"Pokládám dotaz: {query}\n")

# Spuštění řetězce
result = qa_chain(query)

# --- Zobrazení výsledků ---
print("Odpověď modelu:")
print("-" * 50)
print(result['result'].strip())
print("-" * 50)

print("\nZdroje použité pro odpověď:")
for doc in result['source_documents']:
    print(f" - Soubor: {doc.metadata.get('source', 'N/A')}, Strana: {doc.metadata.get('page', 'N/A')}")